In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import pymongo
import cx_Oracle


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [2]:
# df_sgm = pd.read_csv('df_trans_user_item_date.csv', sep='\t', dtype='object')
# df_item = pd.read_csv('MASTER_ITEM_202012121853.csv', dtype='object')
# df_sgm['TR_DATE'] = df_sgm['TR_DATE'].map(lambda x: eval(x))
# df_sgm['TR_PLU'] = df_sgm['TR_PLU'].apply(lambda x: x[0:-1])

In [3]:
conn_str = u'report/justd0it@10.234.152.61:1521/alfabi'
connection = cx_Oracle.connect(conn_str)

conn_mongo = pymongo.MongoClient\
        ("mongodb://127.0.0.1:27017/seq_trans")
mydb = conn_mongo["seq_trans"]
print(mydb.list_collection_names())

mycol = mydb['member_item_seq']

['member_item_seq']


In [4]:
# q_plu = """
# SELECT DISTINCT MPA_PLU, MPA_DESC, MPA_DESC_SUBCAT FROM MASTER_PLU_ALFAGIFT mpa
# where lower(mpa.MPA_DESC) like 'sgm eksplor%'
# """

# df_plu = pd.read_sql_query(q_plu, connection)     
# li_plu = df_plu['MPA_PLU']


# df_all = []
# for plu in li_plu[0:20]:
#     df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
#     df_all.append(df)

# df_seq_1 = pd.concat(df_all)
# df_seq_1.head()
df_seq_1 = pd.DataFrame()

In [5]:
q_plu = """
SELECT DISTINCT MPA_PLU, MPA_DESC, MPA_DESC_SUBCAT FROM MASTER_PLU_ALFAGIFT mpa
where lower(mpa.MPA_DESC) like 'sgm eksplor%'
or lower(mpa.MPA_DESC) like 'anlene%'
or lower(mpa.MPA_DESC) like 'dancow%'
or lower(mpa.MPA_DESC) like 'bebelac%'
or lower(mpa.MPA_DESC) like 'lactogrow%'
or lower(mpa.MPA_DESC) like 'ff baby%'
or lower(mpa.MPA_DESC) like 'pepsodent pg economy%'
"""

df_plu = pd.read_sql_query(q_plu, connection)     
li_plu = df_plu['MPA_PLU']


df_all = []
for plu in li_plu:
    df = pd.DataFrame(mycol.find({'TR_PLU_PCS':plu}))
    df_all.append(df)

df_seq_2 = pd.concat(df_all)
df_seq_2.head()

,_id,TR_MEMBERS,TR_PLU_PCS,updated_at,TRX_DATE,last_order_date,next_order_date
0,5fd90a51b8f8002aaaba19da,9990013140543181,120763,2020-12-21 03:12:38.591,"{'1607904000': 1, '1607558400': 1, '1603929600': 1}",NaT,NaT
1,5fd90a51b8f8002aaaba19e7,9990011494110271,120763,2020-12-21 17:31:38.595,"{'1607904000': 1, '1597276800': 1}",NaT,NaT
2,5fd90a51b8f8002aaaba2495,9990021000743963,120763,2020-12-20 20:31:26.734,{'1607904000': 1},NaT,NaT
3,5fd90a51b8f8002aaaba3bb2,9990011419048278,120763,2020-12-21 06:36:17.581,"{'1607990400': 1, '1607904000': 1, '1605398400': 1, '1602201600': 1}",NaT,NaT
4,5fd90a52b8f8002aaaba5d59,9990012150859676,120763,2020-12-22 13:21:50.268,"{'1607904000': 1, '1603929600': 1, '1594684800': 1, '1591142400': 1, '1588896000': 1, '1588550400': 1, '1588204800': 1, '1588032000': 1, '1587859200': 1, '1587427200': 1, '1586908800': 1}",NaT,NaT


In [6]:
df_seq = pd.concat([df_seq_1, df_seq_2]).reset_index(drop=True)

In [7]:
q_nas = """
SELECT ITEM_N, DESCP, UKURAN, PANJANG, LEBAR, TINGGI, UOM FROM MASTER_NAS_M mnm 
"""

df_nas = pd.read_sql_query(q_nas, connection)
display(df_nas.head())

def norm_uom(uom):
    li_uom_norm = []
    li_weight = []
    
    if uom == np.nan:
        li_weight.append(np.nan)
        li_uom_norm.append(np.nan)
    elif uom == 'KG':
        li_weight.append(1000)
        li_uom_norm.append('G')
    elif uom == 'L':
        li_weight.append(1000)
        li_uom_norm.append('ML')
    else:
        li_weight.append(1)
        li_uom_norm.append(uom)

    return li_uom_norm[0], li_weight[0]
            
        
df_nas['UOM_NORM'], df_nas['weight'] = zip(*df_nas['UOM'].apply(norm_uom))
df_nas['UKURAN_NORM'] = df_nas['UKURAN'] * df_nas['weight']


df_nas['UOM'] = df_nas['UOM_NORM']
df_nas['UKURAN']= df_nas['UKURAN_NORM']

,ITEM_N,DESCP,UKURAN,PANJANG,LEBAR,TINGGI,UOM
0,5440,PEPSODENT MOUTHWASH 300ML,300.0,4.0,7.0,18.5,ML
1,113609,FORMULA S.GIGI S.P SYSTEM MED,0.0,2.0,6.0,23.5,None
2,192612,SMILE UP GEL COOL X'PLOSION 65G,65.0,3.0,17.5,3.7,GR
3,192614,SMILE UP GEL FRESH MINT 65GR,65.0,2.8,16.0,3.8,GR
4,11388,ZWITSAL PASTA G.ANAK ORG 50G,50.0,15.0,3.8,2.8,GR


In [8]:
df = pd.merge(df_seq, df_nas, left_on='TR_PLU_PCS', right_on='ITEM_N')[list(df_seq) + ['DESCP','UKURAN','UOM']]
df = df[df['TRX_DATE'].notnull()]

min_freq = 6
df_hist = df[df['TRX_DATE'].apply(lambda x: len(x)) >= min_freq]

In [9]:
df_hist.head()

,_id,TR_MEMBERS,TR_PLU_PCS,updated_at,TRX_DATE,last_order_date,next_order_date,DESCP,UKURAN,UOM
4,5fd90a52b8f8002aaaba5d59,9990012150859676,120763,2020-12-22 13:21:50.268,"{'1607904000': 1, '1603929600': 1, '1594684800': 1, '1591142400': 1, '1588896000': 1, '1588550400': 1, '1588204800': 1, '1588032000': 1, '1587859200': 1, '1587427200': 1, '1586908800': 1}",NaT,NaT,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
11,5fd90a53b8f8002aaabace0b,9990013066618077,120763,2020-12-20 21:49:50.480,"{'1608336000': 1, '1608076800': 1, '1607904000': 1, '1607817600': 1, '1607731200': 1, '1607126400': 1, '1607040000': 1}",2020-12-19,2020-12-22,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
13,5fd90a54b8f8002aaabb0e0d,9990013033091757,120763,2020-12-22 11:45:37.832,"{'1607904000': 1, '1602547200': 1, '1602460800': 1, '1601942400': 1, '1597536000': 1, '1597449600': 1, '1591228800': 1, '1587859200': 1}",2020-12-14,2021-02-06,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
14,5fd90a54b8f8002aaabb1661,9990011350125734,120763,2020-12-21 18:41:42.311,"{'1608336000': 1, '1607904000': 1, '1607558400': 1, '1606694400': 1, '1606176000': 1, '1603152000': 1, '1602892800': 1, '1601856000': 1, '1601596800': 1, '1600646400': 1, '1598400000': 1, '1596412800': 1}",2020-12-19,2020-12-31,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR
25,5fd90a61b8f8002aaabc4d87,9990013096098148,120763,2021-01-02 15:30:13.096,"{'1607904000': 1, '1606176000': 1, '1606089600': 3, '1605571200': 1, '1605484800': 2, '1605312000': 1, '1605225600': 1, '1604534400': 1, '1608595200': 2, '1608508800': 1, '1608768000': 1, '1609200000': 1, '1609459200': 1}",2021-01-01,2021-01-03,SGM EKSPLOR 1+ VANILA BOX 150G,150.0,GR


In [10]:
# set_time = {'1608163200': 1, '1607904000': 1, '1607040000': 1, '1606176000': 1, '1605916800': 1, '1605657600': 1, '1605225600': 1, '1602288000': 1}
# set_time = sorted(set_time.items())


# print(set_time)

In [11]:
def cal_feature(set_timestamp):

    set_timestamp = sorted(set_timestamp.items())
    last_order = int(set_timestamp[-1][0])
    
    li_seq = []
    qty_seq = []
    for idx in range(0, len(set_timestamp)-1, 1):

        diff = int(set_timestamp[idx+1][0]) - int(set_timestamp[idx][0])
        diff_day = int((diff / 3600) / 24)
        li_seq.append(diff_day)
    
#     read_seq = [datetime.fromtimestamp(int(set_timestamp[i][0])).strftime('%d-%b-%y') for i in range(len(set_timestamp))]
#     qty_seq = [i[1] for i in set_timestamp][-1*(min_freq):-1]

    read_seq = []
    qty_seq = []
    for i in set_timestamp:
        read_seq.append(datetime.fromtimestamp(int(i[0])).strftime('%d%b%y'))
        qty_seq.append(i[1])
        
    qty_seq = qty_seq[-1*(min_freq):-1]
    
    li_seq = li_seq[-1*(min_freq-1):]
    # max diff day
    seq_max = np.max(li_seq)
    # min diff day
    seq_min = np.min(li_seq)
    # day since
    seq_since = li_seq[-2]
    # mean
    seq_mean = round(np.mean(li_seq),2)
    # mean_qty
    seq_mean_qty = round(np.mean([li_seq[i]/qty_seq[i] for i in range(len(li_seq))]),2)
    # variance
    seq_var = round(np.var(li_seq),2)

    ## appending
    li_seq.append(seq_max)
    li_seq.append(seq_min)
    li_seq.append(seq_since)
    li_seq.append(seq_mean)
    li_seq.append(seq_var)
    
    li_seq.extend(qty_seq)
    li_seq.append(seq_mean_qty)


    return li_seq, last_order, read_seq

In [12]:
df_hist['tr_date_seq'], df_hist['last_order'], df_hist['read_seq'] = zip(*df_hist['TRX_DATE'].apply(cal_feature))

<ipython-input-12-dd2dc8b0fd6e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hist['tr_date_seq'], df_hist['last_order'], df_hist['read_seq'] = zip(*df_hist['TRX_DATE'].apply(cal_feature))
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [13]:
#col_feature = ['t_5', 't_4', 't_3', 't_2', 't_1', 'max', 'min', 'since', 'mean', 'variance', 'q_5', 'q_4', 'q_3', 'q_2', 'q_1', 'mean_qty']
col_feature = ['t_'+ str(i) for i in range((min_freq-1),0,-1)] + \
                ['max', 'min', 'since', 'mean', 'variance'] + \
                ['q_'+ str(i) for i in range((min_freq-1),0,-1)] + ['mean_qty']
train = pd.DataFrame(df_hist.tr_date_seq.tolist(), index=df_hist.index, columns=col_feature)

# X = train.drop('t_0', 1)
# Y = train['t_0']

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [14]:
train.head()

,t_5,t_4,t_3,t_2,t_1,max,min,since,mean,variance,q_5,q_4,q_3,q_2,q_1,mean_qty
4,4,26,41,107,46,107,4,107,44.8,1180.56,1,1,1,1,1,44.8
11,7,1,1,2,3,7,1,2,2.8,4.96,1,1,1,1,1,2.8
13,1,51,6,1,62,62,1,1,24.2,710.96,1,1,1,1,1,24.2
14,35,6,10,4,5,35,4,4,12.0,136.40,1,1,1,1,1,12.0
25,7,1,2,5,3,7,1,5,3.6,4.64,1,1,2,1,1,3.4


### Compare time consuming in different netto
> 
* first we user with less variance, we assume user buy based on their consumption and less affected by promotion or random purchase date
* age might be affect consumption, cause more mature, amount of consumption should be decreased by parent

In [15]:
# with open('watchlist_item.txt') as f:
#     watchlist = f.read().splitlines()
#     df_watchlist = pd.DataFrame({'DESCP':watchlist})

In [16]:
train_descp = pd.concat([train,df_hist], 1)[col_feature + ['TR_MEMBERS', 'TR_PLU_PCS', 'DESCP', 'TRX_DATE', 'read_seq', 'UKURAN', 'UOM']]
train_descp = pd.merge(train_descp, df_plu[['MPA_PLU', 'MPA_DESC_SUBCAT']], left_on='TR_PLU_PCS', right_on='MPA_PLU', how='left')
train_descp = train_descp[train_descp['UKURAN'].notnull()]
train_descp['UKURAN'] = train_descp['UKURAN'].astype(int)

In [17]:
# train_descp.groupby('UKURAN').agg({'mean_qty':'mean'})

# train_descp_cond = train_descp[train_descp['variance'] < 100]
# train_descp_cond.groupby('UKURAN').agg({'mean_qty':'mean'})

In [18]:
train_descp['DESCP_TYPE'] = train_descp['DESCP'].str.split(' ').str[0:-3].str.join(' ')

In [19]:
train_descp.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN','UOM']).agg({'mean_qty':'mean'})

mean_qty
MPA_DESC_SUBCAT        DESCP_TYPE       UKURAN UOM           
Perawatan Gigi & Mulut PEPSODENT        75     GR   31.972913
                       PEPSODENT PG     190    GR   27.400701
                                        225    GR   29.760530
Susu Bubuk             ANLENE ACTIFIT   250    GR   28.612279
                       ANLENE GOLD      250    GR   31.883143
                                        650    GR   35.290159
                       ANLENE GOLD+     250    GR   31.852210
                                        650    GR   35.004416
                       BEBELAC 3        200    GR   18.070764
                                        400    GR   19.807894
                                        800    GR   22.273734
                                        1000   G    25.103436
                       BEBELAC 4        400    GR   20.908469
                                        800    GR   23.639949
                                        1000   G    24.417095
                       BEBELAC GOLD 3   700    GR   19.288812
                       DANCOW 1+        200    GR   17.642578
                                        400    GR   19.610957
                                        800    GR   21.354453
                       DANCOW 3+        200    GR   19.165906
                                        400    GR   21.328852
                                        800    GR   23.479656
                       DANCOW 5+        400    GR   22.350404
                                        800    GR   24.724012
                       DANCOW FG        200    GR   24.991475
                                        400    GR   25.274282
                                        800    GR   25.943872
                       FF BABY TAHAP 1  400    GR   12.180741
                                        800    GR   13.331258
                       FF BABY TAHAP 2  200    GR    6.800000
                                        400    GR   12.134538
                                        800    GR   14.225690
                       LACTOGROW 3      180    GR   16.785624
                                        350    GR   19.363391
                                        750    GR   20.951510
                       LACTOGROW 4      750    GR   22.471557
                       SGM EKSPLOR      700    GR   22.754042
                       SGM EKSPLOR 1+   150    GR   20.570908
                                        400    GR   21.765767
                                        600    GR   23.084907
                                        900    GR   23.091402
                       SGM EKSPLOR 3+   150    GR   19.880148
                                        400    GR   21.989024
                                        600    GR   24.037064
                                        900    GR   23.857769
                       SGM EKSPLOR 5+   400    GR   21.021317
                                        900    GR   23.761192
                       SGM EKSPLOR SOYA 700    GR   23.146830
Susu Cair & Es Krim    ANLENE UHT       175    ML    6.097222
                       DANCOW UHT FG    110    ML   21.899963

In [20]:
train_descp_cond = train_descp[train_descp['variance'] < 80]
train_descp_cond.groupby(['MPA_DESC_SUBCAT','DESCP_TYPE','UKURAN','UOM']).agg({'mean_qty':'mean'})

mean_qty
MPA_DESC_SUBCAT        DESCP_TYPE       UKURAN UOM           
Perawatan Gigi & Mulut PEPSODENT        75     GR   16.125065
                       PEPSODENT PG     190    GR    6.033655
                                        225    GR    8.528475
Susu Bubuk             ANLENE ACTIFIT   250    GR   16.636210
                       ANLENE GOLD      250    GR   20.329412
                                        650    GR   24.824056
                       ANLENE GOLD+     250    GR   19.792520
                                        650    GR   24.383785
                       BEBELAC 3        200    GR    8.365978
                                        400    GR    9.835874
                                        800    GR   11.780819
                                        1000   G    12.079328
                       BEBELAC 4        400    GR   10.924481
                                        800    GR   13.232207
                                        1000   G    14.346719
                       BEBELAC GOLD 3   700    GR   11.193049
                       DANCOW 1+        200    GR    8.633450
                                        400    GR    9.809720
                                        800    GR   12.838501
                       DANCOW 3+        200    GR    9.348239
                                        400    GR   11.150172
                                        800    GR   14.195306
                       DANCOW 5+        400    GR   12.421663
                                        800    GR   15.680766
                       DANCOW FG        200    GR   11.649884
                                        400    GR   13.229273
                                        800    GR   15.126856
                       FF BABY TAHAP 1  400    GR    8.615480
                                        800    GR   10.853916
                       FF BABY TAHAP 2  200    GR    6.131915
                                        400    GR    8.372529
                                        800    GR   10.843243
                       LACTOGROW 3      180    GR    7.814107
                                        350    GR    9.120563
                                        750    GR   12.211539
                       LACTOGROW 4      750    GR   13.101636
                       SGM EKSPLOR      700    GR   11.914729
                       SGM EKSPLOR 1+   150    GR    8.851021
                                        400    GR    9.840965
                                        600    GR   10.652210
                                        900    GR   12.083197
                       SGM EKSPLOR 3+   150    GR    8.757089
                                        400    GR   10.972321
                                        600    GR   11.416026
                                        900    GR   13.693104
                       SGM EKSPLOR 5+   400    GR   11.323438
                                        900    GR   14.499203
                       SGM EKSPLOR SOYA 700    GR   11.863417
Susu Cair & Es Krim    ANLENE UHT       175    ML    5.335385
                       DANCOW UHT FG    110    ML    8.084125

In [34]:
from ipywidgets import interact, interact_manual
@interact_manual
def show_articles_more_than(nama = sorted(list(train_descp['DESCP_TYPE'].unique())), 
                            ukuran = sorted(list(train_descp['UKURAN'].unique())),
                            lower=(2, 60, 2),
                            upper=(2, 60, 2)):
    show_data = train_descp.loc[(train_descp['DESCP_TYPE'] == nama) & (train_descp['UKURAN'] == ukuran)\
                                & (train_descp['mean_qty'] >= lower) & (train_descp['mean_qty'] <= upper)]\
                .drop(['since','mean','TR_PLU_PCS','TRX_DATE','MPA_PLU','MPA_DESC_SUBCAT','DESCP_TYPE'],1)
    print(show_data.shape)
    return show_data.sample(10)

interactive(children=(Dropdown(description='nama', options=('ANLENE ACTIFIT', 'ANLENE GOLD', 'ANLENE GOLD+', '…